# Programação com sockets a.k.a. como programar aplicações que usam a rede

Já vimos que existem diversas camadas na pilha de rede TCP/IP.

### Pilha TCP/IP (http://www.tcpipguide.com/free/t_IPDatagramEncapsulation.htm)
![](02_programacao_com_sockets/ip_datagram_encapsulation.png)

Existem, porém, outras famílias de pilhas de rede além da TCP/IP.
O Linux, por exemplo, [suporta as seguintes famílias](https://github.com/torvalds/linux/blob/master/net/socket.c#L170-L217):
- PF_UNIX: comunicação entre processos da mesma máquina
- PF_INET: redes TCP/IP com IPv4
- PF_INET6: redes TCP/IP com IPv6
- PF_BLUETOOTH: redes sem fio Bluetooth
- PF_CAN: redes automotivas e industriais
- PF_PACKET: redes com pacotes RAW (sem nenhum processamento de cabeçalhos). Utilizado para desenvolvimento de protocolos de mais baixo nível e monitoramento.

Os sockets são um conjunto de APIs criados pelo projeto [BSD](https://docs.freebsd.org/en/books/developers-handbook/sockets/).

Seu objetivo é expor uma interface de programação homogênea para os usuários que se utilizavam das diferentes pilhas, independente do sistema operacional executado.

O diagrama abaixo mostra onde exatamente os sockets são posicionados na pilha de rede.

[![](https://mermaid.ink/img/pako:eNqVk99KwzAUxl8l5HqTpupN0cGcwoauFrsh0nqRLUcXtiYhSf3D2MN45YP4Yp7aKZ2bqC2E9jtf8p3-ylnSqRZAI3pvuZmR0UmuCF6unNRCjNWAZLH2MNF6fluXNyzjFA1nzvC3V00EkLEr316s1A1vN0mCrLeQoDxExIF9ADswxyzYY3iHrbWUaOuPWbh_0NgKSuzIPK8yU-k8FJxcGrB8KrXiC3IhVfnU2D666sYpho-FiUiIZzdqA9QHSUTWbbBGadjtBRkuDSnp3wQZLj_1Vn3jUbud6ukcfLvdWSfXhg_nDrSMZKfg5l6b3WTZH8iyLEV6UmgbkUWFRP2HI9vN8VoqoR_dFklWkzSYQL4lDLDYwBlu4mTbONkvONk2TvaFs3bVr71-N47PLrJDMpwY1yIhKT47r_4aHtNZWzb8lV61QVu0AFtwKXAOlpUlp34GBeQ0wkfB7TynuVqhj5dep89qSiNvS2jR0gju4VRyxFnQ6I4vHKogpNd2WA_Wx3yt3gFM9fwv?type=png)](https://mermaid.live/edit#pako:eNqVk99KwzAUxl8l5HqTpupN0cGcwoauFrsh0nqRLUcXtiYhSf3D2MN45YP4Yp7aKZ2bqC2E9jtf8p3-ylnSqRZAI3pvuZmR0UmuCF6unNRCjNWAZLH2MNF6fluXNyzjFA1nzvC3V00EkLEr316s1A1vN0mCrLeQoDxExIF9ADswxyzYY3iHrbWUaOuPWbh_0NgKSuzIPK8yU-k8FJxcGrB8KrXiC3IhVfnU2D666sYpho-FiUiIZzdqA9QHSUTWbbBGadjtBRkuDSnp3wQZLj_1Vn3jUbud6ukcfLvdWSfXhg_nDrSMZKfg5l6b3WTZH8iyLEV6UmgbkUWFRP2HI9vN8VoqoR_dFklWkzSYQL4lDLDYwBlu4mTbONkvONk2TvaFs3bVr71-N47PLrJDMpwY1yIhKT47r_4aHtNZWzb8lV61QVu0AFtwKXAOlpUlp34GBeQ0wkfB7TynuVqhj5dep89qSiNvS2jR0gju4VRyxFnQ6I4vHKogpNd2WA_Wx3yt3gFM9fwv)

Os sockets foram tão bem recebidos que passaram a fazer parte de padrão POSIX. Foram integrados aos mais diversos sistemas operacionais.

Mas como são programados?

### Programando um cliente TCP

Primeiro precisamos importar a biblioteca de sockets

In [62]:
import socket

Depois precisamos criar um socket

In [63]:
# requisita API do SO uma conexão AF_INET (IPV4)
#   com protocolo de transporte SOCK_STREAM (TCP)
socketCliente = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(socketCliente)

<socket.socket fd=1616, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>


Com o socket criado, podemos nos conectar a um servidor.

Mas para nos conectarmos ao servidor, precisamos descobrir seu endereço IP.

In [64]:
ip_servidor = socket.gethostbyname_ex("gnu.org")
print(ip_servidor)

('gnu.org', [], ['209.51.188.116'])


Com o endereço do servidor e sabendo a porta correspondente ao tipo de aplicação que queremos nos conectar,
tentamos nos conectar.

In [65]:
socketCliente.connect((ip_servidor[2][0], 80)) # porta 80 = HTTP
print(socketCliente)

<socket.socket fd=1616, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.0.114', 54376), raddr=('209.51.188.116', 80)>


Se conseguirmos nos conectar, podemos então enviar mensagens para o servidor conectado.

In [66]:
requisicao_http = \
"""GET / HTTP/1.1
Host: gnu.org
Connection: keep-alive
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36
Upgrade-Insecure-Requests: 1
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
Accept-Language: en-US,en;q=0.9
Accept-Encoding: identity"""
res = socketCliente.send(bytes(requisicao_http, "utf-8"))
print(res)

398


Após o envio da nossa requisição HTTP, precisamos aguardar a resposta

In [67]:
resposta_http = socketCliente.recv(10000)
print(resposta_http)

b'HTTP/1.1 400 Bad Request\r\nDate: Mon, 05 Dec 2022 00:43:11 GMT\r\nServer: Apache/2.4.29\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains; preload\r\nX-Frame-Options: sameorigin\r\nX-Content-Type-Options: nosniff\r\nContent-Length: 293\r\nConnection: close\r\nContent-Type: text/html; charset=iso-8859-1\r\n\r\n<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>400 Bad Request</title>\n</head><body>\n<h1>Bad Request</h1>\n<p>Your browser sent a request that this server could not understand.<br />\n</p>\n<hr>\n<address>Apache/2.4.29 Server at nongnu.org Port 80</address>\n</body></html>\n'


Agora que já enviamos uma requisição e recebemos a resposta, podemos fechar a conexão.

In [68]:
res = socketCliente.close()
print(socketCliente)

<socket.socket [closed] fd=-1, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>


### Programando um servidor TCP

Agora que já temos um cliente, precisamos de um servidor.

Como fazer um servidor? Precisamos novamente criar um socket.

In [69]:
socketServidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(socketServidor)

<socket.socket fd=1708, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>


Agora precisamos escutar em uma dada porta, que os clientes conhecem e tentarão se conectar.
Podemos escutar em todas as interfaces de rede do servidor ("0.0.0.0") ou em interfaces específicas.

In [70]:
nome_servidor = socket.gethostname()
print(nome_servidor)
ip_servidor = socket.gethostbyname_ex(nome_servidor)
print(ip_servidor)

DESKTOP-J5V9ICO
('DESKTOP-J5V9ICO', [], ['192.168.56.1', '192.168.0.114'])


Neste caso, escutaremos apenas na interface de rede com o IP acima.

In [71]:
socketServidor.bind((ip_servidor[2][1], 4400))
print(socketServidor)

<socket.socket fd=1708, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.0.114', 4400)>


Após reservamos a porta, podemos começar a escutar.

Precisamos definir o número máximo de conexões simultâneas.

Escutaremos apenas uma por vez.

In [72]:
socketServidor.listen(1)
print(socketServidor)

<socket.socket fd=1708, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.0.114', 4400)>


Agora, configuremos o servidor para esperar por uma conexão.
Se o fizermos agora, sem um cliente, ficaremos bloqueados esperando.
Para solucionar isto, rodaremos um cliente em outra thread.

In [73]:
import time

def cliente():
    socket_cliente_thread = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    time.sleep(2) # coloca a thread para dormir por dois segundos enquanto o servidor é iniciado
    socket_cliente_thread.connect((ip_servidor[2][1], 4400))
    socket_cliente_thread.send(bytes(requisicao_http, "utf-8"))
    msg = socket_cliente_thread.recv(4000)
    print("Cliente:", msg)
    socket_cliente_thread.close()
    print("Cliente:", socket_cliente_thread)

from concurrent.futures import ThreadPoolExecutor
threadPool = ThreadPoolExecutor()
threadPool.submit(cliente)

<Future at 0x24b06c13a90 state=running>

Agora que temos o mesmo cliente pronto para enviar a requisição HTTP e receber a resposta do servidor

In [74]:
(socketParaCliente, enderecoDoCliente) = socketServidor.accept()
print("Servidor:", socketParaCliente)
print("Servidor:", enderecoDoCliente)
time.sleep(1) # coloca o servidor para dormir enquanto o cliente é acordado

Servidor: <socket.socket fd=1716, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.0.114', 4400), raddr=('192.168.0.114', 54378)>
Servidor: ('192.168.0.114', 54378)


Após o cliente se conectar, podemos receber sua requisição. De fato, a requisição é recebida.

In [75]:
msgRecebida = socketParaCliente.recv(4000)
print("Servidor:", msgRecebida)
time.sleep(1) # coloca o servidor para dormir enquanto o cliente é acordado

Servidor: b'GET / HTTP/1.1\nHost: gnu.org\nConnection: keep-alive\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36\nUpgrade-Insecure-Requests: 1\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9\nAccept-Language: en-US,en;q=0.9\nAccept-Encoding: identity'


Também podemos devolver uma resposta. Neste caso, será a própria requisição.

Como pode ser visto abaixo, o cliente de fato recebeu a resposta igual a sua requisição.

Como o clientet só envia e recebe uma mensagem, fecha em seguida seu socket e a thread é finalizada.

In [76]:
socketParaCliente.send(msgRecebida)
time.sleep(1) # coloca o servidor para dormir enquanto o cliente é acordado

Cliente: b'GET / HTTP/1.1\nHost: gnu.org\nConnection: keep-alive\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36\nUpgrade-Insecure-Requests: 1\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9\nAccept-Language: en-US,en;q=0.9\nAccept-Encoding: identity'
Cliente: <socket.socket [closed] fd=-1, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>


Após isto, fechamos a conexão do servidor.

In [77]:
socketServidor.close()
print("Servidor:", socketServidor)

Servidor: <socket.socket [closed] fd=-1, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>
